In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error

# Define quantile loss function
def quantile_loss(y_true, y_pred, quantile=0.9):
    e = y_true - y_pred
    return tf.keras.backend.mean(tf.keras.backend.maximum(quantile * e, (quantile - 1) * e), axis=-1)

# Load data
df = pd.read_csv('IESCO_dataset.csv')
df.dropna(inplace=True)

# Preprocess data
s1 = MinMaxScaler(feature_range=(0,1))
Xs = s1.fit_transform(df)
s2 = MinMaxScaler(feature_range=(0,1))
ys = s2.fit_transform(df[['target_variable']])



# Split data into training and testing sets
X_train1, X_test1, y_train, y_test = train_test_split(Xs, ys, test_size=0.3, shuffle=False)
print('Train set shape', X_train1.shape)
print('testing set shape', X_test1.shape)



In [ ]:
X_train=np.reshape(X_train1,(9189,5,1))
X_test=np.reshape(X_test1,(3939,5,1))

In [ ]:
# Define combined hybrid model
def hybrid_model(input_shape, quantile=0.9):
    inputs = tf.keras.Input(shape=input_shape)
    
    # Shared LSTM layers
    lstm_out = tf.keras.layers.LSTM(64, return_sequences=True)(inputs)
    lstm_out = tf.keras.layers.LSTM(64, return_sequences=True)(inputs)

    
    # Attention mechanism
    attention1 = tf.keras.layers.Attention()([lstm_out, lstm_out])
    attention1 = tf.keras.layers.GlobalAveragePooling1D()(attention1)
    
    # Output layer
    quantile_lstm_out = tf.keras.layers.Dense(1)(attention1)
    
    # Quantile LSTM model
    quantile_lstm_model = tf.keras.Model(inputs=inputs, outputs=quantile_lstm_out)
    quantile_lstm_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
                                loss=lambda y_true, y_pred: quantile_loss(y_true, y_pred, quantile=quantile))
    
    return quantile_lstm_model

# Train the hybrid model
model = hybrid_model((X_train.shape[1], X_train.shape[2]))
history = model.fit(X_train, y_train, epochs=200, batch_size=32)

# Predictions
y_pred = model.predict(X_test)

y_pred = s2.inverse_transform(y_pred)
y_test = s2.inverse_transform(y_test)

# Evaluate the model
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
mae = mean_absolute_error(y_test, y_pred)
mape = mean_absolute_percentage_error(y_test, y_pred)*100
mse = mean_squared_error(y_test, y_pred)
nrmse = (rmse/np.mean(y_pred))*100

print("RMSE:", rmse)
print("MAE:", mae)
print("MAPE:", mape)
print("MSE:", mse)
print("NRMSE:", nrmse)

In [ ]:
y_pred= pd.DataFrame(y_pred)
y_test= pd.DataFrame(y_test)

In [ ]:
result_df = pd.concat([y_pred, y_test], axis=1)
result_df.to_csv(r"C:\Users\hp\Desktop\New folder\IESCO\DAQ-LSTM\DAQ-LSTM_output(4).csv", index=False)